# 

# Automated Optical Inspection at Variass
Help Variass in predicting the defect rates of assembled products based on their specifications

predict the quality (rate of defects per defect type) based on a given Valor code (or list of Valor codes cq the bill of materials of a completely new product)

## Step 1: Introduction

In [3]:
import pandas as pd
import numpy as np

## Step 1 : Introduction
Read excel AOI, order list and VPL (taking around 31s) and converting it to csv for better performance

In [2]:
file_path_order = "data/initial/Order picklists.xlsx"
file_path_vpl = "data/initial/VPL list.xlsx"
pd.read_excel(file_path_order).to_csv("data/orders.csv", index=False)
pd.read_excel(file_path_vpl).to_csv("data/vpl_codes.csv", index=False)

Read the new csv files as three new dataframes 

In [4]:
df_aoi_defects = pd.read_csv("data/initial/AOI defects last year.csv", index_col="id")
df_orders = pd.read_csv("data/orders.csv")
df_vpl = pd.read_csv("data/vpl_codes.csv")

Get de info from the data

In [19]:
print("shape of AOI defects:", df_aoi_defects.shape)
print("shape of orders", df_orders.shape)
print("shape of vpl codes", df_vpl.shape)
print(df_aoi_defects[["reviewed", "defecttypestring", "refid", "partnumber"]].info())

shape of AOI defects: (992952, 61)
shape of orders (68792, 3)
shape of vpl codes (17425, 9)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 992952 entries, 1 to 992956
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   reviewed          992952 non-null  int64 
 1   defecttypestring  987297 non-null  object
 2   refid             992952 non-null  object
 3   partnumber        988647 non-null  object
dtypes: int64(1), object(3)
memory usage: 37.9+ MB
None


,overkill,defecttypestring,moduleid,refid,frameid,windowid,isimportantwindow,partnumber,partid,windowtype,...,framerecty3,framerecty4,defectrectx1,defectrectx2,defectrectx3,defectrectx4,defectrecty1,defectrecty2,defectrecty3,defectrecty4
id,,,,,,,,,,,,,,,,,,,,,
1,0,Missing,0,SC68-P,33,2,0,UHW-00003,5532728F-188C-43fc-97D8-C9FBBE46CF3B,1,...,167846.0,167846.0,499243.0,500321.0,500321.0,499243.0,149534.0,149534.0,150579.0,150579.0
2,0,Missing,0,U1-KD,25,0,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,1,...,149048.0,149048.0,428648.0,436050.0,436050.0,428648.0,114934.0,114934.0,116459.0,116459.0
3,0,Missing,0,U1-KD,25,1,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,5,...,149048.0,149048.0,436675.0,448295.0,448295.0,436675.0,117988.0,117988.0,119084.0,119084.0
4,0,Missing,0,U1-KD,25,2,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,5,...,149048.0,149048.0,424899.0,436519.0,436519.0,424899.0,117991.0,117991.0,119087.0,119087.0
5,0,Missing,0,U1-KD,25,3,0,IR9-00013,6DC3F823-57D2-4817-8CC5-53653A2DD0A0,1,...,149048.0,149048.0,437847.0,440247.0,440247.0,437847.0,120726.0,120726.0,123247.0,123247.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
992952,1,InsufficientSolder,3,TR1,6,2,0,HZZ-00260,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992953,1,InsufficientSolder,4,TR1,6,2,0,HZZ-00260,NaN,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
992954,1,Missing,4,U7,1,7,0,ITI-01426,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_vpl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17425 entries, 0 to 17424
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   partnumber     17425 non-null  object
 1   mpn            17425 non-null  object
 2   VPLpackage     17425 non-null  object
 3   Material type  17425 non-null  object
 4   Position type  17425 non-null  object
 5   Package type   17425 non-null  object
 6   Lead type      17425 non-null  object
 7   Pitch          17425 non-null  object
 8   Subtype        17425 non-null  object
dtypes: object(9)
memory usage: 1.2+ MB


Additional info on defect df 
- *defecttypestring* = Defecttypestring: type of defect
- *reviewd* = 2 after manual inspection the 3D defect is approved, 1 after manual inspection the 3D defect is disapproved (false call)
- *refid* = reference identification with is the position at the PCBA
- *Partnumber* = link to the orderpick list and VPL list

Additional info on order list 
- *Order* = each order corresponds to an part needed for that order and its total amount
- *part number* = each part number corresponds an specific part needed for that specific assembly order
- *total amount* = total amount of parts needed to fulfil the order  

In [5]:
df_merged_order_vpl = pd.merge(df_orders, df_vpl[["partnumber", "VPLpackage"]], left_on="part number", right_on="partnumber")
df_merged_vpl_and_defects = pd.merge(df_vpl[["partnumber", "VPLpackage"]], df_aoi_defects[["reviewed", "defecttypestring", "refid", "partnumber"]], left_on="partnumber", right_on="partnumber")
print("merged orders and vpl")
print(df_merged_order_vpl.head(3))
print("merged vpl and defects")
print(df_merged_vpl_and_defects.head(3))

merged orders and vpl
     Order part number  total amount partnumber              VPLpackage
0  1036933   CAP-00693         504.0  CAP-00693  PZXC-L12/HH-L198W52T53
1  1038499   CAP-00693         200.0  CAP-00693  PZXC-L12/HH-L198W52T53
2  1038502   CAP-00693         200.0  CAP-00693  PZXC-L12/HH-L198W52T53
merged vpl and defects
  partnumber            VPLpackage  reviewed defecttypestring refid
0  VVH-00021  PDSO-J3/XX-L48W46T37         1          Missing    R6
1  VVH-00021  PDSO-J3/XX-L48W46T37         1          Missing    R6
2  VVH-00021  PDSO-J3/XX-L48W46T37         1          Missing    R6


Value counts on the merged df of vpl and defects shows that 

In [6]:
print(df_merged_vpl_and_defects.value_counts(subset=["reviewed"]))
df_merged_vpl_and_defects = df_merged_vpl_and_defects[df_merged_vpl_and_defects["reviewed"] != 0]
df_merged_vpl_and_defects.value_counts(subset=["reviewed"])

reviewed
2           686190
1           228998
0             6134
dtype: int64


reviewed
2           686190
1           228998
dtype: int64

# Step 3 : Defect rates
In  this  step,  you  will  make  descriptive  estimations  on  the  defect  rates  without  considering 
any product specifications.
- (a)  Find the defect rate returned by the automated inspection system on the overall and per defect type.  
- (b)  Find the true- and false-positive rates overall and per defect type, based on the information on both automated and manual inspections

In [1]:
total_amount_parts_ordered = df_orders["total amount"].sum()
# defect rate by the automated inspection system on the overall
defect_rate_overall = (len(df_merged_vpl_and_defects) / total_amount_parts_ordered) * 100
print(defect_rate_overall)
# get all defects 
defect_types = df_merged_vpl_and_defects.value_counts(subset=["defecttypestring"])

# source: https://stackoverflow.com/questions/47136436/python-pandas-convert-value-counts-output-to-dataframe
df_defects = pd.DataFrame(defect_types)
df_defects = df_defects.reset_index()
df_defects.columns = ["defect_type", "amount"]

# calculate the defect rate per defect type
df_defects["defect_rate"] = (df_defects["amount"] / total_amount_parts_ordered) * 100


# Find the true- and false-positive rates overall
true_false_positives_count = df_merged_vpl_and_defects.value_counts(subset=["reviewed"])

df_merged_vpl_and_defects.value_counts(subset=["defecttypestring", "reviewed"]) # way to go for false-positives 


df_defects


NameError: name 'df_orders' is not defined

## Step 3a : Defect amount of the automatic system
Find the defect rate returned by the automated inspection system on the overall and per defect type.

In [58]:
total_nr_of_defects = sum(counts.tolist()[:2])
print("total nr of defects detected by the automatic system (sum of 1 and 2):", total_nr_of_defects)
correct_assemblies = sum(counts.tolist()[2:])
print("correct assemblies:", correct_assemblies)
defect_rate = (total_nr_of_defects / sum(counts.tolist())) * 100
print("Total defect rate detected by the automatic system", round(defect_rate), "%")

total nr of defects detected by the automatic system (sum of 1 and 2): 915188
correct assemblies: 6134
Total defect rate detected by the automatic system 99 %


In [68]:
print("Shape of df_vpl:", df_vpl.shape)
print("Columns names of df_vpl:", df_vpl.columns)
print("Unique number of parts:", df_vpl["partnumber"].nunique())
# a part number can have an different VPLpackage code, so in theory the same part number can have multiple VPLpackage codes
count_partnumbers_vplpackages = df_vpl.value_counts(subset=["partnumber", "VPLpackage"])
is_higher_than_1 = count_partnumbers_vplpackages > 1
print(is_higher_than_1)
print("Num. of rows where the partnumber has the duplicated vpl package:", sum(is_higher_than_1))
df_test_vpl = df_vpl.copy()
df_test_vpl.drop_duplicates(subset=["partnumber", "VPLpackage"], keep="last", inplace=True)
df_test_vpl
# partnumbers_with_dup_vpl = count_partnumbers_vplpackages.index[is_higher_than_1]
# partnumbers_with_dup_vpl

# df_vpl.loc[df_vpl["partnumber", "VPLpackage"].isin(partnumbers_with_dup_vpl), :]


Shape of df_vpl: (17425, 9)
Columns names of df_vpl: Index(['partnumber', 'mpn', 'VPLpackage', 'Material type', 'Position type',
       'Package type', 'Lead type', 'Pitch', 'Subtype'],
      dtype='object')
Unique number of parts: 17286
partnumber  VPLpackage           
XMU-00012   CSXD-R3/QN-L45W20T12      True
DGC-00016   PDSO-G2/XX-L18W13T10      True
KNQ-00004   PDCY-L2/XE-L83W83T105     True
IAD-00256   PDSO-G8/FX-L49W39T16      True
ITI-00146   PDSO-G8/FX-L49W39T18      True
                                     ...  
IN4-00101   PDSO-G5/NT-L21W13T11     False
IN4-00103   PDSO-G24/N2-L78W44T11    False
IN4-00104   PDSO-G16/NX-L50W44T11    False
IN4-00106   PQFP-N49/MX-L70W70T10    False
ZZZ-00024   XDXD-R2/XX-L74W51T7      False
Length: 17329, dtype: bool
Num. of rows where the partnumber has the duplicated vpl package: 96


,partnumber,mpn,VPLpackage,Material type,Position type,Package type,Lead type,Pitch,Subtype
0,VVH-00021,TSM4ZJ202KR10,PDSO-J3/XX-L48W46T37,P,D,SO,J3,X,X
1,INS-00410,LF353M/NOPB,PDSO-G8/FX-L49W39T18,P,D,SO,G8,F,X
2,VWO-00003,82541300,XDXD-R2/XR-L32W16T15,X,D,XD,R2,X,R
3,FMU-00181,NFM3DCC102R1H3L,XXXD-R3/XL-L32W13T7,X,X,XD,R3,X,L
4,KKT-00218,T495X337K010ATE100,PDSO-C2/XX-L73W43T40,P,D,SO,C2,X,X
...,...,...,...,...,...,...,...,...,...
17420,FTV-00129,SLF12565T-220M3R5-PF,XDXD-N2/XI-L125W125T65,X,D,XD,N2,X,I
17421,FCQ-00171,1008CS-101XGLC,XDXD-N2/XI-L29W28T20,X,D,XD,N2,X,I
17422,FCQ-00172,1008CS-330XGLC,XDXD-N2/XI-L29W28T20,X,D,XD,N2,X,I
17423,FM5-00008,TCM2-1T+,XDXX-F6/FT-L38W30T41,X,D,XX,F6,F,T
